# Đặt vấn đề

Bài làm của nhóm em là cào dữ liệu trên trang web [https://batdongsan.vn/ban-nha/](https://batdongsan.vn/ban-nha/), sau đó xử lý dữ liệu và thực hiện dự đoán giá nhà dựa trên các feature mà nhóm tìm thấy.

# Quá trình làm việc

## Thu thập dữ liệu

### Mô tả
Dữ liệu được thu thập bằng cách sử dụng thư viện Selenium. Driver được sử dụng là ChromeDriver 126.0.6478.55.

Bên cạnh đó để tối ưu hóa thời gian cào dữ liệu, em quyết định tích hợp thêm Thread để cào đa luồng.

Nhiệm vụ chính của bước này là thu thập dữ liệu thô từ website để phục vụ cho các quy trình về sau của project.
### Tập tin chính
Source code đảm nhận task này là [parallel_crawling.py](../crawl-data-and-get-coordinates/parallel_crawling.py)

Về quy trình xử lý của script ở từng page:
- Thu thập 20 links trong page hiện tại
- Truy cập vào từng links - bài đăng bán nhà trong page để truy xuất thông tin cần thiết:
    + Title
    + Thông số cơ bản: Diện tích, Số PN, Số WC
    + Mô tả của người đăng

Sau khi kết thúc quy trình ở tất cả các luồng, script sẽ thực hiện concat data và trả về một file csv trong thư mục hiện hữu, bên cạnh đó để phòng trường hợp trục trặc kĩ thuật thì script cũng sẽ thực hiện sao lưu data của từng trang trong thư mục [dataset](../crawl-data-and-get-coordinates/dataset).

Vê các hàm chính trong script:
- `openMultiBrowser` - Mở đồng loạt n trình duyệt để tiến hành chạy đa luồng
- `loadMultiPages` - Ứng với một driver, sẽ load đến đường dẫn cần crawl dữ liệu đầu tiên
- `loadMultiBrowsers` - Tiến hành chạy đa luồng với danh sách các driver đã khởi tạo
- `get_params` - Trích xuất thông tin cần thiết từ mục params trên bài đăng bán nhà (Diện tích, số PN...etc)
- `get_data` - thực hiện cào dữ liệu cho từng luồng
- `runInParallel` - function chính để thực hiện crawl data đa luồng

Để biết thêm chi tiết, người đọc có thể tham khảo phần docstring đã được chú thích trong script.

## Xử lý thêm feature bằng LLM

### Mô tả

Dữ liệu được đưa vào anyscale để chạy lọc ra các thông tin mới từ mô tả của bất động sản. Các thông tin mới đó bao gồm:
- Diện tích phòng khách
- Số lượng phòng ngủ
- Tiện ích xung quanh

Thông tin đưa cho LLM bao gồm dữ liệu phi cấu trúc cột "Description" và cột "Title" sau đó LLM sẽ phân tích thành dữ liệu có cấu trúc. Ví dụ:

```python
"""
+ Thông số cực đẹp 5 x 30m (150m2)
+ Nhà cấp 4, 2PN, toilet, phòng khách, bếp, sân sau trồng cây.
+ Nhà gần sát Mặt tiền, hẻm nhựa xe hơi tránh.
+ Khu vực tập trung nhiều Khu dân cư, các Dự án lớn, thu hút đông đảo các Nhà Đầu Tư lớn, Huyện Nhà Bè hướng đến mục tiêu trở thành Thành phố trực thuộc TP.Hồ Chí Minh.
+ Giá chưa tới 30tr/m2 bao đầu tư.
+ Sổ đang vay bank 2.2 tỷ, pháp lý chuẩn, chủ cần bán gấp.
☎️ Thiện chí alo em Hiền gặp chính chủ thương lượng, miễn trung gian."

Tiêu đề: Bạch Đằng, Quận 1
"""
```

LLM sẽ phân tích và trả về kết quả như sau:

```python
[2, 1, np.NAN, "Bạch Đằng, Quận 1"]
```

Những dữ liệu không tìm được, sẽ trả về `np.NAN`

### Tập tin chính

Source code đảm nhận công việc này là [extract_data.py](../extract_features_from_data/extract_data.py). Trong tập tin này, đảm nhận việc chia nhỏ dữ liệu thô ra thành 10 phần và dùng đa luồng để chia ra xử lý, cuối cùng đóng gói thành 1 file csv duy nhất chứa dữ liệu đã được xử lý.

Trong tập tin này, nó sẽ thực hiện làm sạch dữ liệu thông qua hàm `data_cleaning`, quá trình làm sạch diễn ra như sau: Khử bỏ trùng lặp bằng cách gọi phương thức `drop_duplicates()` của Pandas Dataframe. Sau đấy dùng **regex** 

```python
ad_pattern = r"google|facebook|cửa\wgỗ|cửa\wnhựa|cửa\wsắt|ad|max|marketing|email|sms"
```

để khử bỏ đi những bài đăng chứa quảng cáo trong cột *Description* và *Title*. Kế đến sẽ làm việc với cột *Price* là giá của căn nhà, ta chỉ giữ lại những hàng mà giá chứa chữ "tỷ", thật không may ta cần phải đánh đổi sự mất mát là những căn nhà có giá dưới 1 tỷ đồng nhưng nhờ vậy dữ liệu sẽ chính xác hơn, cuối cùng ta chuyển dữ liệu trong *Price* thành dữ liệu số.

Ngoài ra, `data_cleaning` còn đảm nhận nhiệm vụ khai thác thêm các tính năng mới như "Chỗ đễ xe hơi", "Đang cho thuê", "Tiện ích xung quanh", "Mặt tiền" tất cả đều được xử lý bằng những **regex pattern**. Cuối cùng hàm trả về Pandas Dataframe đã được làm sạch.

Tiếp theo nó chia dữ liệu thành 10 phần để dùng đa luồng phân chia ra xử lý nhằm tăng tốc độ hoàn thành. Mỗi luồng sẽ thực hiện chạy hàm `process(df, start, end)` với các tham số là phần dữ liệu (df), chỉ mục bắt đầu (start) và chỉ mục kết thúc (end), trong hàm này sẽ gọi LLM để xử lý việc trích xuất dữ liệu cuối cùng trả về CSV file có độ dài `end - start` vào thư mục được định sẵn. 

Cuối cùng nó gọi hàm `read_full_data` để nối tất cả các dữ liệu mà đã được chia nhỏ ra xử lý xong thành 1 tập tin csv cuối cùng. Sau đó có thể đem đi qua task tiếp theo.

### Lưu ý thêm

Một phần quan trọng để sử dụng được LLM là API của [anyscale](https://www.anyscale.com/). Hãy tạo tài khoản, sau đó lấy API và dán vào tập tin [private_docu.txt](../extract_features_from_data/private_docu.txt), thật may mắn, anyscale cung cấp API hoàn toàn miễn phí.

# Thực hiện làm sạch dữ liệu

Xử lý các dữ liệu bị thiếu, outlier.

## Thêm các feature mới từ các feature thô ở các bước trước

Bước này liên hệ chặt chẽ với bước làm sạch dữ liệu, vì từ việc feature engineering ta có thể xử lý các feature bị thiếu, và từ các feature cũ ta có thể thêm các feature mới. Các feature được thêm mới là:
- `Cluster`: Phân chia các căn nhà ra thành các cụm
- `Distance to center 0` và `Distance to center 1`: Đo khoảng cách đến trung tâm
Có thể quan sát quá trình feature engineering và EDA trong file [EDA-and-Feature-engineering.ipynb](../EDA_FE/EDA-and-Feature-engineering.ipynb)


# Huấn luyện các mô hình đơn giản

Quá trình này có thể tìm thấy ở file [train.ipynb](../train/train.ipynb)

Nhóm em thực hiện train 2 mô hình đơn giản: Hồi quy tuyến tính và cây quyết định. Kết quả cho thấy:
- Về mặt tuyến tính, dữ liệu không giải thích được nhiều giá của một bất động sản.
- Một số feature không có ý nghĩa lắm đối với cây quyết định, điều đáng cân nhắc khi sử dụng rừng ngẫu nhiên về sau.

Tổng hợp lại, nhóm em đã bỏ 6 cột:

```python
['Cluster', 'For rent', 'Latitude', 'Distance to center 0', 'Distance to center 1', 'Longitude']
```

## Huấn luyện các mô hình phức tạp

Quá trình này gồm 2 file [random_forest.ipynb](../train_complex/random_forest.ipynb) và [pytorch.ipynb](../train_complex/pytorch.ipynb)

Nhóm em thực hiện train 2 mô hình phức tạp, gồm rừng ngẫu nhiên và học sâu:
- Mô hình rừng ngẫu nhiên có r2 là 0.61 và RMSE là 0.62 khi kiểm tra trên tập test.
- Mô hình học sâu có RMSE là 0.66, khi kiểm tra trên tập test.

## Kết luận

Qua quá trình làm việc, nhóm em rút ra các kết luận sau:
- Dữ liệu trên web thiếu rất nhiều dữ liệu cần thiết, từ đó có thể ảnh hưởng rất lớn đến giá nhà của một bất động sản.
- Đôi khi, cách định giá của người bán là không chính xác, và vì thế không nên quá tin tưởng vào giá nhà, mà chỉ xem là thông tin tham khảo.
- Rất nhiều bất động sản bị thiếu hoặc không rõ ràng về địa chỉ.